<a href="https://colab.research.google.com/github/Chandana-Myakala/NLP/blob/main/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load data fromkeras.datasets and perform following computational analysis:- [CO3]
(a) Preprocessing of the Data

(b) Divide data into training and testing data set

(c) Build the Gated Recurrent Units (GRU) Model

(d) Training the GRU Model

(e) Text Generation Using the Trained Model

 (f)  Evaluate Model’s accuracy

In [ ]:
pip install tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
# Step 2: Load and Preprocess Data (using IMDB dataset)
from tensorflow.keras.datasets import imdb

# Set parameters for text processing
vocab_size = 10000  # Restrict vocabulary to 10k most frequent words
max_length = 200  # Maximum length of sequences

# Load IMDB dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform length for all input data
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [ ]:
# Step 3: Build the GRU model
def build_gru_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length=max_length))
    model.add(GRU(128, return_sequences=True))  # First GRU layer with return_sequences
    model.add(GRU(128))  # Second GRU layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    return model

# Initialize GRU model
gru_model = build_gru_model(vocab_size, max_length)

In [ ]:
# Step 4: Train the GRU model
gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_history = gru_model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 554s 1s/step - accuracy: 0.5287 - loss: 0.6963 - val_accuracy: 0.7237 - val_loss: 0.5677
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 518s 1s/step - accuracy: 0.7915 - loss: 0.4544 - val_accuracy: 0.8044 - val_loss: 0.4306
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 562s 1s/step - accuracy: 0.8629 - loss: 0.3273 - val_accuracy: 0.8650 - val_loss: 0.3162


In [ ]:
# Tokenizer to convert sequence back to text
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# Convert X_train from array to list of strings
X_train_str = [' '.join(map(str, sequence)) for sequence in X_train]

# Fit tokenizer on the converted list of strings
tokenizer.fit_on_texts(X_train_str)

In [ ]:
# Step 5: Text Generation using the trained GRU model
def generate_text(model, tokenizer, seed_text, max_sequence_len):
    for _ in range(50):  # Generate 50 words
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == np.argmax(predicted):
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Tokenizer to convert sequence back to text
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# Convert X_train from array to list of strings
X_train_str = [' '.join(map(str, sequence)) for sequence in X_train]

# Fit tokenizer on the converted list of strings
tokenizer.fit_on_texts(X_train_str)

# Generate text using the trained GRU model
seed_text = "The movie was"
generated_text = generate_text(gru_model, tokenizer, seed_text, max_length)
print(f"Generated Text (GRU): {generated_text}")

Generated Text (GRU): The movie was                                                  


In [ ]:
# Step 6: Evaluate GRU model accuracy
gru_test_loss, gru_test_accuracy = gru_model.evaluate(X_test, y_test)
print(f"GRU Model Accuracy: {gru_test_accuracy}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 92s 118ms/step - accuracy: 0.8629 - loss: 0.3215
GRU Model Accuracy: 0.8650400042533875


Compare accuracy of Long sort term memory and Gated recurrent Unit models for text generation using data from tensorflow.keras.datasets. [CO3]

In [ ]:
# Step 7: Build the LSTM Model for comparison
def build_lstm_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length=max_length))
    model.add(LSTM(128, return_sequences=True))  # First LSTM layer with return_sequences
    model.add(LSTM(128))  # Second LSTM layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    return model

# Initialize LSTM model
lstm_model = build_lstm_model(vocab_size, max_length)

# Train the LSTM model
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_history = lstm_model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


391/391 ━━━━━━━━━━━━━━━━━━━━ 512s 1s/step - accuracy: 0.5696 - loss: 0.6669 - val_accuracy: 0.5216 - val_loss: 0.6925
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 493s 1s/step - accuracy: 0.6128 - loss: 0.6347 - val_accuracy: 0.7870 - val_loss: 0.4852
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 498s 1s/step - accuracy: 0.8264 - loss: 0.4202 - val_accuracy: 0.8531 - val_loss: 0.3500


In [ ]:
# Step 8: Evaluate LSTM model accuracy
lstm_test_loss, lstm_test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f"LSTM Model Accuracy: {lstm_test_accuracy}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 178s 228ms/step - accuracy: 0.8517 - loss: 0.3506
LSTM Model Accuracy: 0.8530799746513367


In [ ]:
# Step 9: Compare GRU and LSTM models
print(f"GRU Accuracy: {gru_test_accuracy}, LSTM Accuracy: {lstm_test_accuracy}")

GRU Accuracy: 0.8650400042533875, LSTM Accuracy: 0.8530799746513367
